# # AI for Sustainable Agriculture - Data Cleaning
# **Notebook**: 02_data_cleaning.ipynb  
  
# 
# ## Objectives:
# - Handle missing values (though none were found)
# - Correct data types
# - Remove duplicates
# - Fix inconsistent values
# - Export cleaned data

## Setup  and data loading

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Load raw data
DATA_DIR = Path('../data/raw/')
farmer_raw = pd.read_csv(DATA_DIR / 'farmer_advisor_dataset.csv')
market_raw = pd.read_csv(DATA_DIR / 'market_researcher_dataset.csv')

Farmer Data Cleaning

In [4]:
# Create clean copy
farmer_clean = farmer_raw.copy()

# --- Data Type Corrections ---
farmer_clean['Crop_Type'] = farmer_clean['Crop_Type'].astype('category')

# Convert date if exists
if 'planting_date' in farmer_clean.columns:
    farmer_clean['planting_date'] = pd.to_datetime(farmer_clean['planting_date'])

# --- Handle Placeholder Values ---
# Replace placeholders with NA
placeholders = ['unknown', 'missing', '?', '-', '']
for col in farmer_clean.select_dtypes(include='object'):
    farmer_clean[col] = farmer_clean[col].replace(placeholders, np.nan)

# --- Validate Ranges ---
def clip_outliers(series, min_val, max_val):
    return series.clip(lower=min_val, upper=max_val)

farmer_clean['Soil_pH'] = clip_outliers(farmer_clean['Soil_pH'], 4.0, 9.0)  # Agronomic pH range
farmer_clean['Soil_Moisture'] = clip_outliers(farmer_clean['Soil_Moisture'], 5, 100)  # Percentage scale
farmer_clean['Pesticide_Usage_kg'] = farmer_clean['Pesticide_Usage_kg'].abs()  # Remove negative values

# --- Duplicates ---
print(f"Removed {farmer_clean.duplicated().sum()} duplicate rows")
farmer_clean = farmer_clean.drop_duplicates()


Removed 0 duplicate rows


MArket Data Cleaning

In [5]:
market_clean = market_raw.copy()

# --- Fix Column Names ---
market_clean.columns = market_clean.columns.str.replace(' ', '_').str.lower()

# --- Correct Data Types ---
if 'date' in market_clean.columns:
    market_clean['date'] = pd.to_datetime(market_clean['date'])
    
market_clean['seasonal_factor'] = market_clean['seasonal_factor'].astype('category')

# --- Handle Price Outliers ---
def remove_price_outliers(df):
    q1 = df['market_price_per_ton'].quantile(0.05)
    q3 = df['market_price_per_ton'].quantile(0.95)
    return df[(df['market_price_per_ton'] >= q1) & (df['market_price_per_ton'] <= q3)]

market_clean = remove_price_outliers(market_clean)

Final Visualization

In [6]:
def validate_clean_data(df, df_name):
    print(f"\n=== {df_name} Data Validation ===")
    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nBasic Statistics:")
    display(df.describe(include='all').style.background_gradient(cmap='Blues'))

validate_clean_data(farmer_clean, "Farmer")
validate_clean_data(market_clean, "Market")


=== Farmer Data Validation ===
Rows: 10000, Columns: 10

Missing Values:
Farm_ID                 0
Soil_pH                 0
Soil_Moisture           0
Temperature_C           0
Rainfall_mm             0
Crop_Type               0
Fertilizer_Usage_kg     0
Pesticide_Usage_kg      0
Crop_Yield_ton          0
Sustainability_Score    0
dtype: int64

Basic Statistics:


,Farm_ID,Soil_pH,Soil_Moisture,Temperature_C,Rainfall_mm,Crop_Type,Fertilizer_Usage_kg,Pesticide_Usage_kg,Crop_Yield_ton,Sustainability_Score
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000.000000,10000.000000,10000.000000,10000.000000
unique,nan,nan,nan,nan,nan,4,nan,nan,nan,nan
top,nan,nan,nan,nan,nan,Soybean,nan,nan,nan,nan
freq,nan,nan,nan,nan,nan,2559,nan,nan,nan,nan
mean,5000.500000,6.499494,29.988655,25.027475,174.969854,nan,125.212701,10.521074,5.489634,50.213200
std,2886.895680,0.574181,11.493376,5.769509,72.860989,nan,43.132645,5.535558,2.608809,28.667146
min,1.000000,5.500021,10.002907,15.000186,50.031967,nan,50.007543,1.001370,1.000323,0.003672
25%,2500.750000,6.003992,20.027802,20.078612,111.786631,nan,87.945625,5.675684,3.218402,25.974568
50%,5000.500000,6.495380,29.862527,24.955117,174.468002,nan,125.188012,10.619785,5.490626,50.234210
75%,7500.250000,6.993481,40.052369,30.053313,237.812507,nan,162.619398,15.330758,7.740585,74.938267



=== Market Data Validation ===
Rows: 9000, Columns: 10

Missing Values:
market_id                   0
product                     0
market_price_per_ton        0
demand_index                0
supply_index                0
competitor_price_per_ton    0
economic_indicator          0
weather_impact_score        0
seasonal_factor             0
consumer_trend_index        0
dtype: int64

Basic Statistics:


,market_id,product,market_price_per_ton,demand_index,supply_index,competitor_price_per_ton,economic_indicator,weather_impact_score,seasonal_factor,consumer_trend_index
count,9000.000000,9000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000,9000.000000
unique,nan,4,nan,nan,nan,nan,nan,nan,3,nan
top,nan,Rice,nan,nan,nan,nan,nan,nan,Medium,nan
freq,nan,2304,nan,nan,nan,nan,nan,nan,3027,nan
mean,4988.229444,nan,299.590579,125.192231,125.086341,300.836430,1.001492,50.441645,nan,99.573771
std,2884.010159,nan,104.371987,43.350975,42.902671,115.542341,0.291812,29.183616,nan,28.848305
min,1.000000,nan,119.844972,50.025587,50.024338,100.056917,0.500231,0.037550,nan,50.005314
25%,2480.750000,nan,207.677834,87.822031,88.231536,200.617751,0.748743,24.930568,nan,74.576450
50%,4983.500000,nan,300.389982,124.531063,124.690523,302.621849,1.005346,50.814679,nan,99.426415
75%,7488.250000,nan,389.899693,162.230553,161.554482,402.040012,1.255528,75.831807,nan,124.706106


 Exporting Cleaned Data
 

In [8]:
# %% [markdown]
# ## 5. Export Cleaned Data (CSV Format)

# %%
# Create processed directory if not exists
Path("../data/processed").mkdir(exist_ok=True)

# Save cleaned data as CSV
farmer_clean.to_csv("../data/processed/cleaned_farmer_data.csv", index=False)
market_clean.to_csv("../data/processed/cleaned_market_data.csv", index=False)

print("✅ Data saved as CSV files")

✅ Data saved as CSV files
